In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [4]:
# The number of steps within one time segment
TIME_PERIODS = 15
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 15
N_FEATURES = 10

In [5]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'Class_label']


In [6]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [7]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [8]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [9]:
 pd.options.display.float_format = "{:,.5f}".format

In [10]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [11]:
verbose, epochs, batch_size = 0, 600, 512

In [12]:
# def step_decay(epoch):
#    initial_lrate = 0.1
#    drop = 0.1
#    epochs_drop = 16.0
#    lrate = initial_lrate * tf.math.pow(drop,  
#            tf.math.floor((1+epoch)/epochs_drop))
#    return lrate

In [13]:
# lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [14]:
import os
path='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/sub_wise_process_TT/'

In [16]:
for i in range(1,28):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    # print(y_train)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
    print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 15, 10
    n_depth=1
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(1,9), padding='same', kernel_initializer="he_normal",strides=(3,1), \
                           input_shape=(n_steps,n_length,n_depth)))
    model.add(MaxPooling2D(pool_size=(4,1),strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Conv2D(filters=64, kernel_size=(5,1),padding="same",kernel_initializer="he_normal",strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2093))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='tanh'))
    model.add(Dense(n_outputs, activation='softmax'))
    # adam=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    sgd=optimizers.SGD(learning_rate=1e-2, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/sub_wise_process_TT/S1_tr.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 182000

/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/sub_wise_process_TT/S1_tt.csv
Number of columns in the dataframe: 11
Number of rows in the dataframe: 78000

x_train shape:  (12133, 15, 10)
12133 training samples
y_train shape:  (12133,)
num_time_periods 15
num_sensors 10
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0]
x_train shape: (15, 10)
input_shape: (15, 10)
[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
New y_train sha

21/24 [=========================>....] - ETA: 0s - loss: 3.6444 - accuracy: 0.0791
Epoch 00015: val_accuracy improved from 0.08117 to 0.08329, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
24/24 [==============================] - 0s 13ms/step - loss: 3.6444 - accuracy: 0.0804 - val_loss: 3.6377 - val_accuracy: 0.0833
Epoch 16/600
24/24 [==============================] - ETA: 0s - loss: 3.6314 - accuracy: 0.0817
Epoch 00016: val_accuracy did not improve from 0.08329
24/24 [==============================] - 0s 12ms/step - loss: 3.6314 - accuracy: 0.0817 - val_loss: 3.6238 - val_accuracy: 0.0831
Epoch 17/600
16/24 [===================>..........] - ETA: 0s - loss: 3.6120 - accuracy: 0.0830
Epoch 00017: val_accuracy improved from 0.08329 to 0.08463, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
24/24 [==============================] - 0s 10ms/step - loss: 3.6170 - acc

Epoch 39/600
17/24 [====================>.........] - ETA: 0s - loss: 3.4266 - accuracy: 0.1044
Epoch 00039: val_accuracy did not improve from 0.10406
24/24 [==============================] - 0s 12ms/step - loss: 3.4319 - accuracy: 0.1046 - val_loss: 3.4267 - val_accuracy: 0.1033
Epoch 40/600
17/24 [====================>.........] - ETA: 0s - loss: 3.4252 - accuracy: 0.1062
Epoch 00040: val_accuracy did not improve from 0.10406
24/24 [==============================] - 0s 11ms/step - loss: 3.4264 - accuracy: 0.1062 - val_loss: 3.4239 - val_accuracy: 0.1039
Epoch 41/600
19/24 [======================>.......] - ETA: 0s - loss: 3.4245 - accuracy: 0.1044
Epoch 00041: val_accuracy did not improve from 0.10406
24/24 [==============================] - 0s 11ms/step - loss: 3.4250 - accuracy: 0.1034 - val_loss: 3.4136 - val_accuracy: 0.0973
Epoch 42/600
18/24 [=====================>........] - ETA: 0s - loss: 3.4285 - accuracy: 0.1020
Epoch 00042: val_accuracy did not improve from 0.10406
24/24 

23/24 [===========================>..] - ETA: 0s - loss: 3.3547 - accuracy: 0.1095
Epoch 00066: val_accuracy did not improve from 0.11021
24/24 [==============================] - 0s 12ms/step - loss: 3.3538 - accuracy: 0.1104 - val_loss: 3.3437 - val_accuracy: 0.1058
Epoch 67/600
23/24 [===========================>..] - ETA: 0s - loss: 3.3523 - accuracy: 0.1106
Epoch 00067: val_accuracy did not improve from 0.11021
24/24 [==============================] - 0s 13ms/step - loss: 3.3524 - accuracy: 0.1112 - val_loss: 3.3417 - val_accuracy: 0.1098
Epoch 68/600
22/24 [==========================>...] - ETA: 0s - loss: 3.3480 - accuracy: 0.1117
Epoch 00068: val_accuracy improved from 0.11021 to 0.11406, saving model to /media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/checkpoint.hdf5
24/24 [==============================] - 0s 13ms/step - loss: 3.3483 - accuracy: 0.1116 - val_loss: 3.3438 - val_accuracy: 0.1141
Epoch 69/600
18/24 [=====================>........] - ETA

Epoch 95/600
16/24 [===================>..........] - ETA: 0s - loss: 3.3347 - accuracy: 0.1145
Epoch 00095: val_accuracy did not improve from 0.11637
24/24 [==============================] - 0s 11ms/step - loss: 3.3295 - accuracy: 0.1132 - val_loss: 3.3161 - val_accuracy: 0.1141
Epoch 96/600
18/24 [=====================>........] - ETA: 0s - loss: 3.3372 - accuracy: 0.1121
Epoch 00096: val_accuracy did not improve from 0.11637
24/24 [==============================] - 0s 14ms/step - loss: 3.3296 - accuracy: 0.1156 - val_loss: 3.3120 - val_accuracy: 0.1137
Epoch 97/600
22/24 [==========================>...] - ETA: 0s - loss: 3.3148 - accuracy: 0.1157
Epoch 00097: val_accuracy did not improve from 0.11637
24/24 [==============================] - 0s 12ms/step - loss: 3.3105 - accuracy: 0.1174 - val_loss: 3.3035 - val_accuracy: 0.1119
Epoch 98/600
22/24 [==========================>...] - ETA: 0s - loss: 3.3046 - accuracy: 0.1204
Epoch 00098: val_accuracy did not improve from 0.11637
24/24 

...

Epoch 352/600
19/24 [======================>.......] - ETA: 0s - loss: 3.1273 - accuracy: 0.1409
Epoch 00352: val_accuracy did not improve from 0.13733
24/24 [==============================] - 0s 9ms/step - loss: 3.1330 - accuracy: 0.1402 - val_loss: 3.2216 - val_accuracy: 0.1179
Epoch 353/600
19/24 [======================>.......] - ETA: 0s - loss: 3.1307 - accuracy: 0.1457
Epoch 00353: val_accuracy did not improve from 0.13733
24/24 [==============================] - 0s 10ms/step - loss: 3.1362 - accuracy: 0.1445 - val_loss: 3.1748 - val_accuracy: 0.1360
Epoch 354/600
20/24 [========================>.....] - ETA: 0s - loss: 3.1130 - accuracy: 0.1414
Epoch 00354: val_accuracy did not improve from 0.13733
24/24 [==============================] - 0s 10ms/step - loss: 3.1116 - accuracy: 0.1422 - val_loss: 3.1567 - val_accuracy: 0.1360
Epoch 355/600
19/24 [======================>.......] - ETA: 0s - loss: 3.1255 - accuracy: 0.1430
Epoch 00355: val_accuracy did not improve from 0.13733
24/

23/24 [===========================>..] - ETA: 0s - loss: 3.1095 - accuracy: 0.1481
Epoch 00380: val_accuracy did not improve from 0.14041
24/24 [==============================] - 0s 11ms/step - loss: 3.1098 - accuracy: 0.1483 - val_loss: 3.1734 - val_accuracy: 0.1323
Epoch 381/600
19/24 [======================>.......] - ETA: 0s - loss: 3.0921 - accuracy: 0.1434
Epoch 00381: val_accuracy did not improve from 0.14041
24/24 [==============================] - 0s 13ms/step - loss: 3.0970 - accuracy: 0.1435 - val_loss: 3.1347 - val_accuracy: 0.1356
Epoch 382/600
23/24 [===========================>..] - ETA: 0s - loss: 3.0923 - accuracy: 0.1506
Epoch 00382: val_accuracy did not improve from 0.14041
24/24 [==============================] - 0s 12ms/step - loss: 3.0930 - accuracy: 0.1499 - val_loss: 3.1869 - val_accuracy: 0.1298
Epoch 383/600
23/24 [===========================>..] - ETA: 0s - loss: 3.1188 - accuracy: 0.1428
Epoch 00383: val_accuracy did not improve from 0.14041
24/24 [=========

Epoch 409/600
24/24 [==============================] - ETA: 0s - loss: 3.0932 - accuracy: 0.1485
Epoch 00409: val_accuracy did not improve from 0.14310
24/24 [==============================] - 0s 14ms/step - loss: 3.0932 - accuracy: 0.1485 - val_loss: 3.1253 - val_accuracy: 0.1383
Epoch 410/600
17/24 [====================>.........] - ETA: 0s - loss: 3.1094 - accuracy: 0.1442
Epoch 00410: val_accuracy did not improve from 0.14310
24/24 [==============================] - 0s 12ms/step - loss: 3.1128 - accuracy: 0.1432 - val_loss: 3.1756 - val_accuracy: 0.1196
Epoch 411/600
17/24 [====================>.........] - ETA: 0s - loss: 3.0960 - accuracy: 0.1472
Epoch 00411: val_accuracy did not improve from 0.14310
24/24 [==============================] - 0s 12ms/step - loss: 3.0953 - accuracy: 0.1470 - val_loss: 3.1465 - val_accuracy: 0.1310
Epoch 412/600
17/24 [====================>.........] - ETA: 0s - loss: 3.0705 - accuracy: 0.1497
Epoch 00412: val_accuracy did not improve from 0.14310
24

23/24 [===========================>..] - ETA: 0s - loss: 3.0655 - accuracy: 0.1506
Epoch 00438: val_accuracy did not improve from 0.14310
24/24 [==============================] - 0s 12ms/step - loss: 3.0678 - accuracy: 0.1507 - val_loss: 3.1342 - val_accuracy: 0.1414
Epoch 439/600
18/24 [=====================>........] - ETA: 0s - loss: 3.0724 - accuracy: 0.1509

KeyboardInterrupt: 

In [ ]:
statistics.mean(test_acc)